# Model Based Machine Learning Project

We have chosen to work with a dataset on Life Expectancy collected by (WHO).

## Loadning and visualizing the data 

We start by importing packages.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import statsmodels.api as sm
import random

#Plotting
import matplotlib as mplib
from matplotlib import pyplot as plt
import seaborn as sns
import itertools
palette = itertools.cycle(sns.color_palette("tab20", n_colors=20))
import matplotlib.cm as cm

#Machine learning 
from sklearn import linear_model
import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (6, 4)

In [ ]:
#Loading the data
data = pd.read_csv("archive/Data_processed_subregions.csv",delimiter=";")
#Formatting into pandas dataframe 
LED = pd.DataFrame(data)

In [ ]:
print(LED.info()) #Check quality of data. 

## Setting up data for Pyro model

In [ ]:
LED.head(5) #Display first 5 row to get idea of the data. 

In [ ]:
mat = LED.values #Convert dataframe into matrix 

X = mat[:,6:].astype("float") #These are the regressors
print(X.shape)

y_original = mat[:,3].astype("float") #This is the target variable life expectancy
print(y_original.shape)

#These are the hierarchies
continent = mat[:,1] 
subregion = mat[:,2] 
developed = mat[:,5]
print(continent.shape)

In [ ]:
#Setup split for traning, validation and test data set. We will use 2000-2012 as traning and 2013-2015 as test. 
#Remember to add in report: We can not split randomly as then we will feed info to the model I should not have. 
training_idx = (data['Year'] >= 2000) & (data['Year'] <= 2011)
test_idx = (data['Year'] >= 2012) & (data['Year'] <= 2013)
val_idx = (data['Year'] >= 2014) & (data['Year'] <= 2015)

In [ ]:
# standardize input features
X_mean = X[training_idx,:].mean(axis=0)
X_std = X[training_idx,:].std(axis=0)
X = (X - X_mean) / X_std

# standardize target
y_mean = y_original[training_idx].mean()
y_std = y_original[training_idx].std()
y = (y_original - y_mean) / y_std

In [ ]:
#Plot target variable before and after standardization 
plt.plot(y_original, label='True Values')
plt.plot(y, label='Standardized values')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
def dataSplit(data,trainIDX,testIDX,valIDX): #Define function to split data
    train = data[trainIDX]
    test  = data[testIDX]
    val   = data[valIDX]
    
    return train, test, val

#Split features 
X_train = X[training_idx,:]
X_test = X[test_idx,:]
X_val = X[val_idx, :]

#Split observations
y_train, y_test, y_val = dataSplit(y,training_idx,test_idx,val_idx)

print("num train: %d" % len(y_train))
print("num test: %d" % len(y_test))
print("num val: %d" % len(y_val))

developed_train, developed_test, developed_val = dataSplit(developed,training_idx,test_idx,val_idx)

In [ ]:
# Setting up hierarchical dictonaries 

#Contienent dictonaries
continent_dict = {'Asia': 0, 'Europe': 1, 'Africa': 2, 'North America': 3,  'South America': 4, 'Oceania': 5}

#Sub-region dictonaries
subregion_dict = region_dict = {
    'Southern Asia': 0,
    'Southern Europe': 1,
    'Northern Africa': 2,
    'Sub-Saharan Africa': 3,
    'Latin America and the Caribbean': 4,
    'Western Asia': 5,
    'Australia and New Zealand': 6,
    'Western Europe': 7,
    'Eastern Europe': 8,
    'South-eastern Asia': 9,
    'Northern America': 10,
    'Eastern Asia': 11,
    'Northern Europe': 12,
    'Melanesia': 13,
    'Central Asia': 14,
    'Micronesia': 15,
    'Polynesia': 16
}

#Split hierarchical features 
continent_train =  np.array(LED.loc[training_idx, 'continent'].map(continent_dict))
continent_test  =  np.array(LED.loc[test_idx,     'continent'].map(continent_dict))
continent_val   =  np.array(LED.loc[val_idx,      'continent'].map(continent_dict))

subregion_train =  np.array(LED.loc[training_idx, 'subregion'].map(subregion_dict))
subregion_test  =  np.array(LED.loc[test_idx,     'subregion'].map(subregion_dict))
subregion_val   =  np.array(LED.loc[val_idx,      'subregion'].map(subregion_dict))

## Two-level hierarichal model (Report model)

In this section we define the two-level hierarichical model which is presented in the report. Both alpha and beta are on the second level.

In [ ]:
def hierarchical_model_TwoLevel_AB(X,continents,developed,num_hir, obs=None):
    n_ind = num_hir
    n_dev = 2
    
    alpha_mu = pyro.sample("alpha_mu", dist.Normal(0., 1.))        # Hyper-Prior for the bias mean
    alpha_sigma  = pyro.sample("alpha_sigma", dist.HalfCauchy(2.)) # Hyper-Prior for the bias standard deviation
    
    beta_mu = pyro.sample("beta_mu", dist.Normal(0., 1.))          # Hyper-Prior for the bias mean
    beta_sigma  = pyro.sample("beta_sigma", dist.HalfCauchy(2.))   # Hyper-Prior for the bias standard deviation

    sigma = pyro.sample("sigma", dist.HalfCauchy(2.))              # Prior for the observation variance

    with pyro.plate("developed", n_ind):
        
        with pyro.plate("continents", n_dev):                      # Drawing parameters for each of the groupings
            alpha = pyro.sample("alpha", dist.Normal(alpha_mu, alpha_sigma).to_event()) 
            beta = pyro.sample("beta", dist.Normal(beta_mu*torch.ones(X.shape[1]), 
                                               beta_sigma*torch.ones(X.shape[1])).to_event(1)) 

    with pyro.plate("data"):
        mu = alpha[developed,continents] + torch.sum(torch.mul(X,beta[developed,continents]),dim=1)
        y = pyro.sample("y", dist.Normal(mu,sigma).to_event(), obs=obs)
        
    return y

In [ ]:
#Convert to torch tensors for MCMC
developed_train_pyro = torch.tensor(developed_train.astype("long")).long()  
developed_test_pyro = torch.tensor(developed_test.astype("long")).long()
developed_val_pyro = torch.tensor(developed_val.astype("long")).long()

X_train_pyro = torch.tensor(X_train).float()
y_train_pyro = torch.tensor(y_train).float()

subregion_train_pyro = torch.tensor(subregion_train).long()

# Run inference in Pyro
nuts_kernel = NUTS(hierarchical_model_TwoLevel_AB)
mcmcTwoLevel_AB = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200, num_chains=1)
mcmcTwoLevel_AB.run(X_train_pyro,subregion_train_pyro,developed_train_pyro,17, y_train_pyro)

# Show summary of inference results
mcmcTwoLevel_AB.summary()

In [ ]:
posterior_samples_twoLevel_AB = mcmcTwoLevel_AB.get_samples()

beta_hat_TwoLevel_AB =torch.mean(posterior_samples_twoLevel_AB["beta"], axis=0)    #Use mean approximation to get parameters 
alpha_hat_TwoLevel_AB =torch.mean(posterior_samples_twoLevel_AB["alpha"], axis=0)

#### Plotting traceplots for beta

In [ ]:
plt.rcParams['figure.figsize'] = (10, 10)

for i in range(17): #Looking at traceplot for beta parameters [Change the second index to switch between developed/developing]
    plt.subplot(9,2,i+1)
    plt.plot(posterior_samples_twoLevel_AB["beta"][::2,1,i,:])

#### Plotting traceplots for alpha

In [ ]:
plt.rcParams['figure.figsize'] = (10, 10)

for i in range(17): #Looking at traceplot for alpha parameters [Change the second index to switch between developed/developing]
    plt.subplot(9,2,i+1)
    plt.plot(posterior_samples_twoLevel_AB["alpha"][::2,1,i])

#### Looking into the autocorrelation to validate thinning

In [ ]:
#Looking a autocorrelation without thinning
fig, axs = plt.subplots(9,2, figsize=(8, 20))

# Iterate over data and create ACF plots in subplots
for i in range(17):    
    row = i // 2; col = i % 2
    
    ax = axs[row, col]
    sm.graphics.tsa.plot_acf(posterior_samples_twoLevel_AB["beta"][:,1,i,1], ax=ax, lags=10) #Plot ACF
    ax.set_xlabel("Lag")

plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
#Looking a autocorrelation with thinning
fig, axs = plt.subplots(9,2, figsize=(8, 20))

# Iterate over data and create ACF plots in subplots
for i in range(17):
    row = i // 2; col = i % 2
    
    ax = axs[row, col]
    sm.graphics.tsa.plot_acf(posterior_samples_twoLevel_AB["beta"][::2,1,i,1], ax=ax, lags=10) #Plot ACF
    ax.set_xlabel("Lag")

plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
rand1 = random.randint(0, 1)  #Get random parameter to shown in the report. 
rand2 = random.randint(0, 16)
rand3 = random.randint(0, 14)

fig, axs = plt.subplots(1,2, figsize=(5, 2))  #Define figure 

ax = axs[0] #Without thinning
sm.graphics.tsa.plot_acf(posterior_samples_twoLevel_AB["beta"][:,rand1,rand2,rand3], ax=ax, lags=10) #Plot ACF
ax.set_xlabel("Lag")
ax.set_title("Before thinning")

ax = axs[1] #With thinning
sm.graphics.tsa.plot_acf(posterior_samples_twoLevel_AB["beta"][::2,rand1,rand2,rand3], ax=ax, lags=10) #Plot ACF
ax.set_xlabel("Lag")
ax.set_title("After thinning")

fig.tight_layout()
fig.show()

fig.savefig('ACF_plot.eps',format='eps',transparent=True)

In [ ]:
#Calculating RMSE 
beta_hat_TwoLevel_AB =torch.mean(posterior_samples_twoLevel_AB["beta"][::2], axis=0)
alpha_hat_TwoLevel_AB =torch.mean(posterior_samples_twoLevel_AB["alpha"][::2], axis=0)

def ErrorTestFun_Model4(X_values,cont_idx,dev_idx,y_values):   #Define function to evaluate estimed parameters of the model
    
    multi_row = np.multiply(X_values,beta_hat_TwoLevel_AB[dev_idx,cont_idx,:])  #Perform linear algebra 
    beta_part = torch.sum(multi_row,dim=1)

    y_hat = alpha_hat_TwoLevel_AB[dev_idx,cont_idx] + beta_part
    
    preds = y_hat.numpy() * y_std + y_mean        #Convert back to original domain
    y_true = y_values * y_std + y_mean

    rmse = np.sqrt(np.mean((preds - y_true)**2))  #Calculate RMSE
    
    return rmse
    
rmse_train = ErrorTestFun_Model4(X_train, subregion_train,developed_train_pyro, y_train)
rmse_test  = ErrorTestFun_Model4(X_test, subregion_test, developed_test_pyro, y_test)
rmse_val   = ErrorTestFun_Model4(X_val, subregion_val, developed_val_pyro,  y_val)

print("RMSE train: %.3f \nRMSE test: %.3f \nRMSE val: %.3f" % (rmse_train,rmse_test,rmse_val))

In [ ]:
#Plotting Beta parameters for the Developing grouping 
list_labels = LED.columns.tolist()
num_betavalues = 15

heatMap_Beta_0 = beta_hat_TwoLevel_AB[0, :, :].numpy()

plt.imshow(heatMap_Beta_0,cmap='viridis') #Plot the values in a heatmap
plt.grid(visible=False)
plt.xticks(range(num_betavalues), list_labels[6:], rotation=90)
plt.yticks(list(subregion_dict.values()),list(subregion_dict.keys()))
plt.title("Developing")

for i in range(heatMap_Beta_0.shape[0]): #Add the number in the heatmap
    for j in range(heatMap_Beta_0.shape[1]):
        plt.text(j, i, f'{heatMap_Beta_0[i, j]:.2f}', ha='center', va='center', color='black')

plt.tight_layout()
plt.savefig("BetaMatrixDev0.eps",format="eps")

In [ ]:
#Plotting Beta parameters for the Developed grouping 
heatMap_Beta_1 = beta_hat_TwoLevel_AB[1, :, :].numpy()

plt.imshow(heatMap_Beta_1,cmap='viridis') #Plot the values in a heatmap
plt.grid(visible=False)
plt.xticks(range(num_betavalues), list_labels[6:], rotation=90)
plt.yticks(list(subregion_dict.values()),list(subregion_dict.keys()))
plt.title("Developed")

for i in range(heatMap_Beta_1.shape[0]): #Add the number in the heatmap
    for j in range(heatMap_Beta_1.shape[1]):
        plt.text(j, i, f'{heatMap_Beta_1[i, j]:.2f}', ha='center', va='center', color='black')

plt.tight_layout()
plt.savefig("BetaMatrixDev1.eps",format="eps")

In [ ]:
#Calculating to 90%CI for each of the alpha parameters
ci_lower_dev1 = np.array([])
ci_upper_dev1 = np.array([])

for i in range(17): #For developed sub-regions
    ci_lower_dev1 = np.append(ci_lower_dev1, np.percentile(posterior_samples_twoLevel_AB["alpha"][::2,1,i], (100 - 90) / 2))
    ci_upper_dev1 = np.append(ci_upper_dev1, np.percentile(posterior_samples_twoLevel_AB["alpha"][::2,1,i], 100 - (100 - 90) / 2))
    
ci_lower_dev0 = np.array([])
ci_upper_dev0 = np.array([])

for i in range(17): #For developing sub-regions
    ci_lower_dev0 = np.append(ci_lower_dev0, np.percentile(posterior_samples_twoLevel_AB["alpha"][::2,0,i], (100 - 90) / 2))
    ci_upper_dev0 = np.append(ci_upper_dev0, np.percentile(posterior_samples_twoLevel_AB["alpha"][::2,0,i], 100 - (100 - 90) / 2))

In [ ]:
#Plot the alpha parameter estimates with a 90% CI 
plt.figure(figsize=(10, 5))

plt.subplot(2,1,1) #Alpha parameters for developed sub-regions 
plt.fill_between(list(subregion_dict.values()),ci_lower_dev1,ci_upper_dev1,alpha=0.3,label="Developed",color="lightblue")
plt.plot(alpha_hat_TwoLevel_AB[1, :],linestyle="--",color="black",linewidth=1.0)
plt.xticks(np.arange(17))
plt.tick_params(axis='x', labelbottom=False)
plt.legend()

plt.subplot(2,1,2) #Alpha parameters for developing sub-regions 
plt.fill_between(list(subregion_dict.values()),ci_lower_dev0,ci_upper_dev0,alpha=0.3,label="Developing",color="lightcoral")
plt.plot(alpha_hat_TwoLevel_AB[0, :],linestyle="--",color="black",linewidth=1.0)
plt.xticks(list(subregion_dict.values()), list(subregion_dict.keys()))
plt.xticks(rotation=90)
plt.legend()

plt.tight_layout()

plt.savefig('alpha_par_res.eps',format='eps')
plt.show()